In [2]:
import pandas as pd
from astropy.io import fits
import numpy as np
from pathlib import Path
import glob




In [ ]:
# MUST BE RUN IN ASTROCONDA38

from fast_combine import fast_process, marckie_slice_combine_reduce

In [4]:
%mkdir processed
%mkdir processed_uncorrected

dark_hdu = fits.open(Path('./calibs/Master_DarkBias.fits'))
#SuperDarkBias = dark_hdu[0].data
SuperDarkBias = dark_hdu[0].data.astype(np.float32)


flatC_norm_hdu = fits.open(Path('./calibs/Master_Flat_C_norm.fits'))
SuperNormFlat_C = flatC_norm_hdu[0].data.astype(np.float32)


mkdir: processed: File exists
mkdir: processed_uncorrected: File exists


In [4]:
fast_process(glob.glob('MBAFiles/*/*'), './processed/', exp_time=10, chunk_size=6,
    darkbias=SuperDarkBias, flat=SuperNormFlat_C, do_offset=True
    )

starting process
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_1.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_0.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_1.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_0.fits
combined_2022-11-12T07:22:56.0000__Brorfelde_4_20221111_224933_1.fits
reducing image 0 from file Brorfelde_4_20221111_224933_1.fits with timestamp offset 2000
Successfully sliced and combined combined_2022-11-12T07:22:56.0000__Brorfelde_4_20221111_224933_1.fits
combined_2022-11-12T03:11:08.0000__Brorfelde_1_20221111_183745_1.fits
reducing image 0 from file Brorfelde_1_20221111_183745_1.fits with timestamp offset 2000
combined_2022-11-1

In [3]:
fast_process(glob.glob('MBAFiles/*/*'), './processed_uncorrected/', exp_time=10, chunk_size=6,
    darkbias=0, flat=1, do_offset=True
    )

starting process
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_0.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_1.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_1.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_0.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_1.fits
combined_2022-11-12T04:27:31.0000__Brorfelde_2_20221111_195408_1.fits
reducing image 0 from file Brorfelde_2_20221111_195408_1.fits with timestamp offset 2000
combined_2022-11-12T03:11:08.0000__Brorfelde_1_20221111_183745_1.fits
reducing image 0 from file Brorfelde_1_20221111_183745_1.fits with timestamp offset 2000
combined_2022-11-12T05:46:20.0000__Brorfelde_3_20221111_211257_1.fits
reducing image 0 from file Brorfelde_3_20221111_211

In [ ]:
%mkdir combined_all

fast_process(glob.glob('MBAFiles/*/*'), './combined_all/', exp_time=10, chunk_size=6,
    darkbias=SuperDarkBias, flat=SuperNormFlat_C, do_offset=True
    )

In [4]:
# MUST BE RUN IN (astroconda3)

import alipy
from datetime import datetime

from dateutil.parser import parse


In [ ]:
%mkdir aligned

def align_imgs(images_to_align, ref_image):

    identifications = alipy.ident.run(ref_image, images_to_align, visu=False)

    outputshape = alipy.align.shape(ref_image)

    for id in identifications: # list of the same length as images_to_align.
        if id.ok == True:
            alipy.align.affineremap(id.ukn.filepath, id.trans,
                shape=outputshape,
                makepng=True,
                outdir=f'aligned/')

            print("SUCCESS: %20s : %20s, flux ratio %.2f" % (id.ukn.name,
                                                        id.trans,
                                                        id.medfluxratio))

        else:
            print("FAILED: %20s" % (id.ukn.name))
    return identifications

In [14]:
def _get_time(fp:str):
    name = Path(fp).name.split('__')[0].lstrip('combined_')
    return parse(name)
    #return datetime.datetime.strptime(name, 'combined_%Y-%m-%dT%X.0000')


imgs = pd.DataFrame(
    columns=['filepath', 'timestamp', 'ref_img'],
    dtype=[str, datetime, str]
)

imgs['filepath'] = glob.glob(f"processed/*.fits")

imgs['timestamp'] = imgs['filepath'].apply(_get_time)

night_1_group_1 = imgs['timestamp'] <= datetime.fromisoformat('2022-11-11T04:49:35')

imgs[night_1_group_1]['ref_img'] = 'CHANGE THIS'

night_2 = (
    (datetime.fromisoformat('2022-11-12T00:00:00') <= imgs['timestamp']) & 
    (imgs['timestamp'] < datetime.fromisoformat('2022-11-13T00:00:00'))
    )

imgs[night_2]['ref_img'] = 'CHANGE THIS'

night_3 = (
    (datetime.fromisoformat('2022-11-13T00:00:00') <= imgs['timestamp']) & 
    (imgs['timestamp'] < datetime.fromisoformat('2022-11-14T00:00:00'))
    )

imgs[night_3]['ref_img'] = 

# times = {datetime.datetime.strptime()}
# combine_imgs()

0      False
1       True
2      False
3      False
4      False
       ...  
656    False
657    False
658    False
659    False
660    False
Name: timestamp, Length: 661, dtype: bool

In [7]:
datetime.now().isoformat()

'2022-11-30T13:34:11.092118'